In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

from scipy.stats import pearsonr

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#reading in train.csv of datasets

df_pre_initial_working_data=pd.read_csv('../datasets/train.csv')

In [3]:
df_initial_working_data=pd.DataFrame.copy(df_pre_initial_working_data)

In [4]:
# correcting typo in an entry found later when doing joint plot:
df_initial_working_data.loc[1699,'Garage Yr Blt']=2007



In [5]:
#printing shape and head for initial look and size of dataframe

print(df_initial_working_data.shape)
df_initial_working_data.head()

(2051, 81)


,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [6]:
features=['1st Flr SF','2nd Flr SF','MS SubClass','MS Zoning','Lot Frontage','Lot Area','Neighborhood','Overall Qual','Year Built','Roof Style','Mas Vnr Type','Exter Qual','Foundation','Bsmt Qual','Bsmt Cond','Bsmt Exposure','BsmtFin Type 1','BsmtFin SF 1','Total Bsmt SF','Heating QC','Gr Liv Area','Kitchen Qual','TotRms AbvGrd','Garage Type','Garage Yr Blt','Garage Finish','Garage Cars','Garage Area','SalePrice']

In [7]:
df_features_pre_clean=df_initial_working_data[features]

In [8]:
len(features)

29

In [9]:
# function to clean 'Nominal' variables

def clean_nominal_func(df_to_be_cleaned,variable_colname_as_str,drop_nan_yes_or_no):
    

    df_clean_nominal=pd.DataFrame.copy(df_to_be_cleaned)

    # counting nan
    print(df_clean_nominal.isnull().sum())
    
    if (df_clean_nominal[variable_colname_as_str].isnull().sum())==0:
        print(f"There are no entries with null values in '{variable_colname_as_str}'.\n")
        
        # displaying value_counts
        print("These are the value_counts.\n")
        print(df_clean_nominal[variable_colname_as_str].value_counts())
        
    elif (df_clean_nominal[variable_colname_as_str].isnull().sum())!=0:
        print(f"There are some entries with null values in '{variable_colname_as_str}'.\n")   
        
        # if drop_nan_yes_or_no is 'Yes', to drop all rows with NaN values,
        if drop_nan_yes_or_no == 'Yes' or drop_nan_yes_or_no == 'yes':
            if (df_clean_nominal[variable_colname_as_str].isnull().sum())!=0:
                df_clean_nominal.dropna(subset=[variable_colname_as_str],inplace=True)
        
        # if drop_nan_yes_or_no is 'No', to replace nan values with 'Unknown'
        elif drop_nan_yes_or_no == 'No' or drop_nan_yes_or_no == 'no':
            # replacing nan values with 'Unknown'
            df_clean_nominal[variable_colname_as_str].fillna('Unknown', inplace=True)
            
        # post nan processing printout to verify
        print('\nAfter null value processing, this is the new null count.\n')
        print(df_clean_nominal.isnull().sum())
        print('\nAfter null value processing, these are the new value_counts.\n')
        print(df_clean_nominal[variable_colname_as_str].value_counts())

    
    # converting nominal columns with int or float values into str
    if type(df_clean_nominal[variable_colname_as_str]) != str:
        df_clean_nominal=pd.DataFrame(df_clean_nominal[variable_colname_as_str].map(lambda x: str(x)))

    # converting a categorical column into a one-hot encoded matrix; making subset df of dummy variables 
    dummy_var_df=pd.get_dummies(df_clean_nominal[[variable_colname_as_str]])
    
    # concating nordinal values with rest of df
    df_clean_nominal_return=pd.concat([dummy_var_df,df_to_be_cleaned.drop(columns=[variable_colname_as_str])],axis='columns')

    # containing converted ordinal values with rest of df
    return (df_clean_nominal_return)

In [10]:
# function for cleaning 'Ordinal' variables

def clean_ordinal_func (df_to_be_cleaned,variable_colname_as_str,drop_nan_yes_or_no,ordinal_string):
    
    df_clean_ordinal=pd.DataFrame.copy(df_to_be_cleaned)


    # counting nan
    print(df_clean_ordinal.isnull().sum())
    
    if (df_clean_ordinal[variable_colname_as_str].isnull().sum())==0:
        print(f"There are no entries with null values in '{variable_colname_as_str}'.\n")
        
        # displaying value_counts
        print("These are the value_counts.\n")
        print(df_clean_ordinal[variable_colname_as_str].value_counts())
    
    elif (df_clean_ordinal[variable_colname_as_str].isnull().sum())!=0:
        print(f"There are some entries with null values in '{variable_colname_as_str}'.\n")
        
        # if drop_nan_yes_or_no is 'Yes', to drop all rows with NaN values, 
        if drop_nan_yes_or_no == 'Yes' or drop_nan_yes_or_no == 'yes':
            if (df_clean_ordinal[variable_colname_as_str].isnull().sum())!=0:
                df_clean_ordinal.dropna(subset=[variable_colname_as_str],inplace=True)
                
                
        # if drop_nan_yes_or_no is 'No', to replace nan values with 'Unknown'
        elif drop_nan_yes_or_no == 'No' or drop_nan_yes_or_no == 'no':
            df_clean_ordinal[variable_colname_as_str].fillna('Unknown', inplace=True)

        # post nan processing printout to verify
        print('\nAfter null value processing, this is the new null count.\n')
        print(df_clean_ordinal.isnull().sum())
        print('\nAfter null value processing, these are the new value_counts.\n')
        print(df_clean_ordinal[variable_colname_as_str].value_counts())

    # printing set of all possible values just to check
    print(f"\nSet of all possible values for column {set(df_clean_ordinal[variable_colname_as_str])}\n")

    # initialising list starting with element Unknown
    list_of_possible_column_values=[]

    # extending list with split of string keyed in for full range of possible ordinal str values
    list_of_possible_column_values.extend(ordinal_string.split(','))

    # showing list of all possible column values
    print(f"\nList of all possible values for column, including 'Unknown' for '{variable_colname_as_str}': {list_of_possible_column_values}\n")

    # converting str values in column 'variable_colname_as_str' to integer ordered values
    df_ordinal_values_converted=pd.DataFrame(df_clean_ordinal[variable_colname_as_str].map(lambda x: list_of_possible_column_values.index(x)))

    # printing unconverted and converted value_counts to check conversion accuracy
    print(df_clean_ordinal[variable_colname_as_str].value_counts())
    print(df_ordinal_values_converted[variable_colname_as_str].value_counts())

 
    # concating converted ordinal values with rest of df
    df_clean_ordinal_return=pd.concat([df_ordinal_values_converted,df_to_be_cleaned.drop(columns=[variable_colname_as_str])],axis='columns')

    # containing converted ordinal values with rest of df
    return(df_clean_ordinal_return)

In [11]:
# function for cleaning 'Continuous' or 'Discrete' variables

def clean_condis_func(df_to_be_cleaned,variable_colname_as_str,drop_nan_yes_or_no):
    

    df_clean_condis=pd.DataFrame.copy(df_to_be_cleaned)

    # counting nan
    print(df_clean_condis.isnull().sum())
    
    if (df_clean_condis[variable_colname_as_str].isnull().sum())==0:
        print(f"There are no entries with null values in '{variable_colname_as_str}'.\n")
    
        # displaying value_counts
        print("These are the value_counts.\n")
        print(df_clean_condis[variable_colname_as_str].value_counts())
    
    elif (df_clean_condis[variable_colname_as_str].isnull().sum())!=0:
        print(f"There are some entries with null values in '{variable_colname_as_str}'.\n")
    
        # if drop_nan_yes_or_no is 'Yes', to drop all rows with NaN values, 
        if drop_nan_yes_or_no == 'Yes' or drop_nan_yes_or_no == 'yes':
            if (df_clean_condis[variable_colname_as_str].isnull().sum())!=0:
                df_clean_condis.dropna(subset=[variable_colname_as_str],inplace=True)
                
        # if drop_nan_yes_or_no is 'No', to replace NaN values with 'Unknown'
        elif drop_nan_yes_or_no == 'No' or drop_nan_yes_or_no == 'no':
            df_clean_condis[variable_colname_as_str].fillna('Unknown', inplace=True)
        
        # post nan processing printout to verify
        print('\nAfter null value processing, this is the new null count.\n')
        print(df_clean_condis.isnull().sum())
        print('\nAfter null value processing, these are the new value_counts.\n')
        print(df_clean_condis[variable_colname_as_str].value_counts())

    return (df_clean_condis)

## ANS:

For MS Subclass, no null values, generally poor correlation with SalePrice.
Only MS Subclass_60 has a 0.36 correlation to it.

Using only MS Subclass_60, it has 394 out of 2051 entries. 19.2%.
MS Subclass_50, correlation -0.18, 198/2051 entries, 9.7%.

In [12]:
# calling cleaning functions

df_under_cleaning=clean_nominal_func (df_features_pre_clean,'MS SubClass','Yes')

print(df_under_cleaning.shape)
df_under_cleaning.head()

1st Flr SF          0
2nd Flr SF          0
MS SubClass         0
MS Zoning           0
Lot Frontage      330
Lot Area            0
Neighborhood        0
Overall Qual        0
Year Built          0
Roof Style          0
Mas Vnr Type       22
Exter Qual          0
Foundation          0
Bsmt Qual          55
Bsmt Cond          55
Bsmt Exposure      58
BsmtFin Type 1     55
BsmtFin SF 1        1
Total Bsmt SF       1
Heating QC          0
Gr Liv Area         0
Kitchen Qual        0
TotRms AbvGrd       0
Garage Type       113
Garage Yr Blt     114
Garage Finish     114
Garage Cars         1
Garage Area         1
SalePrice           0
dtype: int64
There are no entries with null values in 'MS SubClass'.

These are the value_counts.

20     770
60     394
50     198
120    132
30     101
70      90
160     88
80      86
90      75
190     46
85      28
75      16
45      11
180     11
40       4
150      1
Name: MS SubClass, dtype: int64
(2051, 44)


,MS SubClass_120,MS SubClass_150,MS SubClass_160,MS SubClass_180,MS SubClass_190,MS SubClass_20,MS SubClass_30,MS SubClass_40,MS SubClass_45,MS SubClass_50,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,0,0,0,0,0,0,0,0,0,0,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,0,0,0,0,0,0,0,0,0,0,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,0,0,0,0,0,1,0,0,0,0,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,0,0,0,0,0,0,0,0,0,0,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,0,0,0,0,0,0,0,0,0,1,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


In [13]:
# further cleaning MS SubClass

df_under_cleaning.drop(columns=['MS SubClass_120', 'MS SubClass_150', 'MS SubClass_160', 'MS SubClass_180',
                                'MS SubClass_190', 'MS SubClass_20', 'MS SubClass_30', 'MS SubClass_40',
                                'MS SubClass_45', 'MS SubClass_70','MS SubClass_75', 'MS SubClass_80',
                                'MS SubClass_85', 'MS SubClass_90'],inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()

(2051, 30)


,MS SubClass_50,MS SubClass_60,1st Flr SF,2nd Flr SF,MS Zoning,Lot Frontage,Lot Area,Neighborhood,Overall Qual,Year Built,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,0,1,725,754,RL,NaN,13517,Sawyer,6,1976,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,0,1,913,1209,RL,43.0,11492,SawyerW,7,1996,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,0,0,1057,0,RL,68.0,7922,NAmes,5,1953,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,0,1,744,700,RL,73.0,9802,Timber,5,2006,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,1,0,831,614,RL,82.0,14235,SawyerW,6,1900,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

For MS Zoning, no null values, poor correlation with SalePrice, based on my assignment of values. Note high percentage of 6 which corresponds to Residential Low Density. 1598/2051 entries. 77.9%.

Readjusted assignment of values from 'Unknown,I (all),A (agr),FV,RH,RM,RL,RP,C (all)'
to 'Unknown,RP,I (all),C (all),A (agr),RH,RM,FV,RL'

Fit seemed better.

## Using MS Zoning as ordinal based on order: 'Unknown,RP,I (all),C (all),A (agr),RH,RM,FV,RL'.

Apply to holdout data.

In [14]:
# calling cleaning functions

df_under_cleaning=clean_ordinal_func (df_under_cleaning,'MS Zoning','Yes','Unknown,RP,I (all),C (all),A (agr),RH,RM,FV,RL')

print(df_under_cleaning.shape)
df_under_cleaning.head()

MS SubClass_50      0
MS SubClass_60      0
1st Flr SF          0
2nd Flr SF          0
MS Zoning           0
Lot Frontage      330
Lot Area            0
Neighborhood        0
Overall Qual        0
Year Built          0
Roof Style          0
Mas Vnr Type       22
Exter Qual          0
Foundation          0
Bsmt Qual          55
Bsmt Cond          55
Bsmt Exposure      58
BsmtFin Type 1     55
BsmtFin SF 1        1
Total Bsmt SF       1
Heating QC          0
Gr Liv Area         0
Kitchen Qual        0
TotRms AbvGrd       0
Garage Type       113
Garage Yr Blt     114
Garage Finish     114
Garage Cars         1
Garage Area         1
SalePrice           0
dtype: int64
There are no entries with null values in 'MS Zoning'.

These are the value_counts.

RL         1598
RM          316
FV          101
C (all)      19
RH           14
A (agr)       2
I (all)       1
Name: MS Zoning, dtype: int64

Set of all possible values for column {'I (all)', 'FV', 'RL', 'RH', 'A (agr)', 'C (all)', 'RM'}


Li

,MS Zoning,MS SubClass_50,MS SubClass_60,1st Flr SF,2nd Flr SF,Lot Frontage,Lot Area,Neighborhood,Overall Qual,Year Built,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,8,0,1,725,754,NaN,13517,Sawyer,6,1976,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,8,0,1,913,1209,43.0,11492,SawyerW,7,1996,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,8,0,0,1057,0,68.0,7922,NAmes,5,1953,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,8,0,1,744,700,73.0,9802,Timber,5,2006,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,8,1,0,831,614,82.0,14235,SawyerW,6,1900,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

For Lot Frontage, there are 330 null entries.

Correlation seems visually good, 0.34, may have some outliers.

Normal distribution of data.

## Use Lot Frontage.

Has null values in holdout data.

Need to apply same imputation for NaN entries in test and holdout.




In [15]:
# calling cleaning functions

df_under_cleaning=clean_condis_func (df_under_cleaning,'Lot Frontage','No')

print(df_under_cleaning.shape)
df_under_cleaning.head()

MS Zoning           0
MS SubClass_50      0
MS SubClass_60      0
1st Flr SF          0
2nd Flr SF          0
Lot Frontage      330
Lot Area            0
Neighborhood        0
Overall Qual        0
Year Built          0
Roof Style          0
Mas Vnr Type       22
Exter Qual          0
Foundation          0
Bsmt Qual          55
Bsmt Cond          55
Bsmt Exposure      58
BsmtFin Type 1     55
BsmtFin SF 1        1
Total Bsmt SF       1
Heating QC          0
Gr Liv Area         0
Kitchen Qual        0
TotRms AbvGrd       0
Garage Type       113
Garage Yr Blt     114
Garage Finish     114
Garage Cars         1
Garage Area         1
SalePrice           0
dtype: int64
There are some entries with null values in 'Lot Frontage'.


After null value processing, this is the new null count.

MS Zoning           0
MS SubClass_50      0
MS SubClass_60      0
1st Flr SF          0
2nd Flr SF          0
Lot Frontage        0
Lot Area            0
Neighborhood        0
Overall Qual        0
Year Built

,MS Zoning,MS SubClass_50,MS SubClass_60,1st Flr SF,2nd Flr SF,Lot Frontage,Lot Area,Neighborhood,Overall Qual,Year Built,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,8,0,1,725,754,Unknown,13517,Sawyer,6,1976,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,8,0,1,913,1209,43,11492,SawyerW,7,1996,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,8,0,0,1057,0,68,7922,NAmes,5,1953,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,8,0,1,744,700,73,9802,Timber,5,2006,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,8,1,0,831,614,82,14235,SawyerW,6,1900,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


In [16]:
# further cleaning Lot Frontage's Unknown values

# considered inputing a SLR on Lot Frontage and SalePrice, but due to time constraints, will just
# impute a simple SalePrice.mean and Lot Frontage.mean correlation.


temp_df=df_initial_working_data['Lot Frontage'].dropna()
temp_df.mean()

#simple SalePrice.mean and Lot Frontage.mean correlation.
simple_ratio=df_initial_working_data['SalePrice'].mean()/temp_df.mean()

df_under_cleaning['Lot Frontage'].replace('Unknown', (df_under_cleaning['SalePrice']/simple_ratio),inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()


(2051, 30)


,MS Zoning,MS SubClass_50,MS SubClass_60,1st Flr SF,2nd Flr SF,Lot Frontage,Lot Area,Neighborhood,Overall Qual,Year Built,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,8,0,1,725,754,49.65955,13517,Sawyer,6,1976,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,8,0,1,913,1209,43.00000,11492,SawyerW,7,1996,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,8,0,0,1057,0,68.00000,7922,NAmes,5,1953,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,8,0,1,744,700,73.00000,9802,Timber,5,2006,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,8,1,0,831,614,82.00000,14235,SawyerW,6,1900,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

For Lot Area, there are no null entries.

Correlation seems visually good, 0.3, may have some outliers, there seem to be two linear relationships,
one for houses between 0 and 30,000 sqft, and another for 30,000 sqft to 70,000.
Beyond that few houses, and they can be seen as outliers. Consider dropping these.

## Use Lot Area.

Do feature engineering to split into 2 columns for houses of 2 size ranges. Likely to get higher correlation too. Unsure how to actually code it efficiently. If this is the only one that needs splitting,
split train and holdout data along these ranges, get two models, then combine back results, if time permits, else let the model iterate best fit parameters and coefficients.

Need to apply same imputation for NaN entries in test and holdout.

Due to contraints, will just use the data as it is.


## ANS:

For Neighborhood, there are no null entries.

Distribution of entries seems balanced across the sub-categories.

High correlation of 0.45 for NoRidge
and 0.26 for NW_Ames and Somerst.

Their representation is also roughly median, so unlikely to be outliers.

## Use Neighborhood NoRidge, NW_Ames and Somerst.

In [17]:
# calling cleaning functions

df_under_cleaning=clean_nominal_func (df_under_cleaning,'Neighborhood','Yes')

print(df_under_cleaning.shape)
df_under_cleaning.head()

MS Zoning           0
MS SubClass_50      0
MS SubClass_60      0
1st Flr SF          0
2nd Flr SF          0
Lot Frontage        0
Lot Area            0
Neighborhood        0
Overall Qual        0
Year Built          0
Roof Style          0
Mas Vnr Type       22
Exter Qual          0
Foundation          0
Bsmt Qual          55
Bsmt Cond          55
Bsmt Exposure      58
BsmtFin Type 1     55
BsmtFin SF 1        1
Total Bsmt SF       1
Heating QC          0
Gr Liv Area         0
Kitchen Qual        0
TotRms AbvGrd       0
Garage Type       113
Garage Yr Blt     114
Garage Finish     114
Garage Cars         1
Garage Area         1
SalePrice           0
dtype: int64
There are no entries with null values in 'Neighborhood'.

These are the value_counts.

NAmes      310
CollgCr    180
OldTown    163
Edwards    143
Somerst    130
NridgHt    122
Gilbert    116
Sawyer     111
NWAmes      87
SawyerW     87
Mitchel     82
BrkSide     76
Crawfor     71
IDOTRR      69
Timber      48
NoRidge     48


,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,0,0,0,0,0,0,0,0,0,0,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,0,0,0,0,0,0,0,0,0,0,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,0,0,0,0,0,0,0,0,0,0,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,0,0,0,0,0,0,0,0,0,0,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,0,0,0,0,0,0,0,0,0,0,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


In [18]:
# further cleaning Neighborhood

df_under_cleaning.drop(columns=['Neighborhood_Blmngtn', 'Neighborhood_Blueste',
       'Neighborhood_BrDale', 'Neighborhood_BrkSide',
       'Neighborhood_ClearCr', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards',
       'Neighborhood_Gilbert', 'Neighborhood_Greens',
       'Neighborhood_GrnHill', 'Neighborhood_IDOTRR',
       'Neighborhood_Landmrk', 'Neighborhood_MeadowV',
       'Neighborhood_Mitchel', 'Neighborhood_NPkVill', 
       'Neighborhood_NWAmes', 'Neighborhood_NridgHt',
       'Neighborhood_OldTown', 'Neighborhood_SWISU',
       'Neighborhood_Sawyer', 'Neighborhood_SawyerW',
       'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker'],inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()




(2051, 32)


,Neighborhood_NAmes,Neighborhood_NoRidge,Neighborhood_Somerst,MS Zoning,MS SubClass_50,MS SubClass_60,1st Flr SF,2nd Flr SF,Lot Frontage,Lot Area,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,0,0,0,8,0,1,725,754,49.65955,13517,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,0,0,0,8,0,1,913,1209,43.00000,11492,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,1,0,0,8,0,0,1057,0,68.00000,7922,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,0,0,0,8,0,1,744,700,73.00000,9802,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,0,0,0,8,1,0,831,614,82.00000,14235,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

For Overall Qual, there are no null entries.

Very strong correlation of 0.8, with pseudo-normal like distribution.

## Use Overall Qual as Predictor.

## ANS:

For Year Built, there are no null entries.

Good correlation of 0.57, wide distribution of values.

## Use Year Built as Predictor.

## ANS:

For Roof Style, there are no null entries.



1619/2051 entries are Gable, with a -0.25 correlation.

397/2051 entries are Hip, with 0.27 correlation.


## Use Roof Style Gable and Hip as predictors.

In [19]:
# calling cleaning functions

df_under_cleaning=clean_nominal_func (df_under_cleaning,'Roof Style','Yes')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
1st Flr SF                0
2nd Flr SF                0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Roof Style                0
Mas Vnr Type             22
Exter Qual                0
Foundation                0
Bsmt Qual                55
Bsmt Cond                55
Bsmt Exposure            58
BsmtFin Type 1           55
BsmtFin SF 1              1
Total Bsmt SF             1
Heating QC                0
Gr Liv Area               0
Kitchen Qual              0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
dtype: int64
There are no entries with null values in 'Roof Style'.

These are the value_counts.

Gable 

,Roof Style_Flat,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Roof Style_Shed,Neighborhood_NAmes,Neighborhood_NoRidge,Neighborhood_Somerst,MS Zoning,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,0,1,0,0,0,0,0,0,0,8,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,0,1,0,0,0,0,0,0,0,8,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,0,1,0,0,0,0,1,0,0,8,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,0,1,0,0,0,0,0,0,0,8,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,0,1,0,0,0,0,0,0,0,8,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


In [20]:
# further cleaning Roof Style

df_under_cleaning.drop(columns=['Roof Style_Flat', 'Roof Style_Gambrel',
                                'Roof Style_Mansard', 'Roof Style_Shed'],inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()



(2051, 33)


,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,Neighborhood_NoRidge,Neighborhood_Somerst,MS Zoning,MS SubClass_50,MS SubClass_60,1st Flr SF,2nd Flr SF,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,1,0,0,0,0,8,0,1,725,754,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,1,0,0,0,0,8,0,1,913,1209,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,1,0,1,0,0,8,0,0,1057,0,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,1,0,0,0,0,8,0,1,744,700,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,1,0,0,0,0,8,1,0,831,614,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

For Exter Qual, there are 0 null entries.

1247/2051 are TA
697 are Gd

Distribution is mainly on 2 peaks, with good correlation of 0.71.

## Use Exter Qual as Predictor.

In [21]:
# calling cleaning functions

df_under_cleaning=clean_ordinal_func (df_under_cleaning,'Exter Qual','Yes','Unknown,Po,Fa,TA,Gd,Ex')

print(df_under_cleaning.shape)
df_under_cleaning.head()


Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
1st Flr SF                0
2nd Flr SF                0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
Exter Qual                0
Foundation                0
Bsmt Qual                55
Bsmt Cond                55
Bsmt Exposure            58
BsmtFin Type 1           55
BsmtFin SF 1              1
Total Bsmt SF             1
Heating QC                0
Gr Liv Area               0
Kitchen Qual              0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
dtype: int64
There are no entries with null values in 'Exter Qual'.

These a

,Exter Qual,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,Neighborhood_NoRidge,Neighborhood_Somerst,MS Zoning,MS SubClass_50,MS SubClass_60,1st Flr SF,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,4,1,0,0,0,0,8,0,1,725,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,4,1,0,0,0,0,8,0,1,913,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,3,1,0,1,0,0,8,0,0,1057,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,3,1,0,0,0,0,8,0,1,744,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,3,1,0,0,0,0,8,1,0,831,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

For Foundation, there are 0 null entries.

Distribution is mainly on 2 catgories
926/2051 for PConc correlation good at 0.53.
863/2051 for CBlock correlation decent at -0.36.

## use Foundation PConc and CBlock for Predictors.

In [22]:
# calling cleaning functions

df_under_cleaning=clean_nominal_func (df_under_cleaning,'Foundation','Yes')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
1st Flr SF                0
2nd Flr SF                0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
Foundation                0
Bsmt Qual                55
Bsmt Cond                55
Bsmt Exposure            58
BsmtFin Type 1           55
BsmtFin SF 1              1
Total Bsmt SF             1
Heating QC                0
Gr Liv Area               0
Kitchen Qual              0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
dtype: int64
There are no entries with null values in 'Foundation'.

These a

,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Exter Qual,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,0,1,0,0,0,0,4,1,0,0,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,0,0,1,0,0,0,4,1,0,0,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,0,1,0,0,0,0,3,1,0,1,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,0,0,1,0,0,0,3,1,0,0,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,0,0,1,0,0,0,3,1,0,0,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


In [23]:
# further cleaning Roof Style

df_under_cleaning.drop(columns=['Foundation_BrkTil', 'Foundation_Slab',
                                'Foundation_Stone', 'Foundation_Wood'],inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()



(2051, 34)


,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,Neighborhood_NoRidge,Neighborhood_Somerst,MS Zoning,MS SubClass_50,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,1,0,4,1,0,0,0,0,8,0,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,0,1,4,1,0,0,0,0,8,0,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,1,0,3,1,0,1,0,0,8,0,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,0,1,3,1,0,0,0,0,8,0,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,0,1,3,1,0,0,0,0,8,1,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

For Bsmt Qual, there are 55 null entries. The null entries are actually houses with no Basements, hence no Bsmt Qual.

As correlation for dropping all 55 null entries is 0.68 and assigning a value of 0 to the null entries results in correlation of 0.61, the difference is not too much, and due to time constraints, the treatment for Bsmt Qual will be to assign value of 0 to all null entries.

0.68 is a good correlation score.

## Use Bsmt Qual as Predictor, all Null values to be converted to 0.

In [24]:
# calling cleaning functions

df_under_cleaning=clean_ordinal_func (df_under_cleaning,'Bsmt Qual','No','Unknown,Po,Fa,TA,Gd,Ex')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Foundation_CBlock         0
Foundation_PConc          0
Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
1st Flr SF                0
2nd Flr SF                0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
Bsmt Qual                55
Bsmt Cond                55
Bsmt Exposure            58
BsmtFin Type 1           55
BsmtFin SF 1              1
Total Bsmt SF             1
Heating QC                0
Gr Liv Area               0
Kitchen Qual              0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
dtype: int64
There are some entries with null va

,Bsmt Qual,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,Neighborhood_NoRidge,Neighborhood_Somerst,MS Zoning,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,3,1,0,4,1,0,0,0,0,8,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,4,0,1,4,1,0,0,0,0,8,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,3,1,0,3,1,0,1,0,0,8,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,4,0,1,3,1,0,0,0,0,8,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,2,0,1,3,1,0,0,0,0,8,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

For Bsmt Cond, there are 55 null entries. The null entries are actually houses with no Basements, hence no Bsmt Cond.

As correlation for dropping all 55 null entries is 0.18 and assigning a value of 0 to the null entries results in correlation of 0.23, the difference is not too much, and due to time constraints, the treatment for Bsmt Cond will be to assign value of 0 to all null entries.

Visual EDA indicates that Bsmt Cond can be considered as predictor.

## Use Bsmt Cond as Predictor, all Null values to be converted to 0.

In [25]:
# calling cleaning functions

df_under_cleaning=clean_ordinal_func (df_under_cleaning,'Bsmt Cond','No','Unknown,Po,Fa,TA,Gd,Ex')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Bsmt Qual                 0
Foundation_CBlock         0
Foundation_PConc          0
Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
1st Flr SF                0
2nd Flr SF                0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
Bsmt Cond                55
Bsmt Exposure            58
BsmtFin Type 1           55
BsmtFin SF 1              1
Total Bsmt SF             1
Heating QC                0
Gr Liv Area               0
Kitchen Qual              0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
dtype: int64
There are some entries with null va

,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,Neighborhood_NoRidge,Neighborhood_Somerst,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,3,3,1,0,4,1,0,0,0,0,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,3,4,0,1,4,1,0,0,0,0,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,3,3,1,0,3,1,0,1,0,0,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,3,4,0,1,3,1,0,0,0,0,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,4,2,0,1,3,1,0,0,0,0,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

For Bsmt Exposure, there are 58 null entries. The null entries are actually houses with no Basements, hence no Bsmt Exposure.

As correlation for dropping all 58 null entries is 0.41 and assigning a value of 0 to the null entries results in correlation of 0.42, the difference is not too much, and due to time constraints, the treatment for Bsmt Exposure will be to assign value of 0 to all null entries.

Visual EDA indicates that Bsmt Exposure can be considered as predictor.

## Use Bsmt Exposure as Predictor, all Null values to be converted to 0.

In [26]:
# calling cleaning functions

df_under_cleaning=clean_ordinal_func (df_under_cleaning,'Bsmt Exposure','No','Unknown,No,Mn,Av,Gd')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Bsmt Cond                 0
Bsmt Qual                 0
Foundation_CBlock         0
Foundation_PConc          0
Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
1st Flr SF                0
2nd Flr SF                0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
Bsmt Exposure            58
BsmtFin Type 1           55
BsmtFin SF 1              1
Total Bsmt SF             1
Heating QC                0
Gr Liv Area               0
Kitchen Qual              0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
dtype: int64
There are some entries with null va

,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,Neighborhood_NoRidge,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,1,3,3,1,0,4,1,0,0,0,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,1,3,4,0,1,4,1,0,0,0,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,1,3,3,1,0,3,1,0,1,0,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,1,3,4,0,1,3,1,0,0,0,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,1,4,2,0,1,3,1,0,0,0,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

For BsmtFin Type 1, there are 55 null entries. The null entries are actually houses with no Basements, hence no BsmtFin Type 1.

As correlation for dropping all 55 null entries is 0.32 and assigning a value of 0 to the null entries results in correlation of 0.35, the difference is not too much, and due to time constraints, the treatment for Bsmt Exposure will be to assign value of 0 to all null entries.

Visual EDA indicates that BsmtFin Type 1 can be considered as predictor.

## Use BsmtFin Type 1 as Predictor, all Null values to be converted to 0.

In [27]:
# calling cleaning functions

df_under_cleaning=clean_ordinal_func (df_under_cleaning,'BsmtFin Type 1','No','Unknown,Unf,LwQ,Rec,BLQ,ALQ,GLQ')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Bsmt Exposure             0
Bsmt Cond                 0
Bsmt Qual                 0
Foundation_CBlock         0
Foundation_PConc          0
Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
1st Flr SF                0
2nd Flr SF                0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
BsmtFin Type 1           55
BsmtFin SF 1              1
Total Bsmt SF             1
Heating QC                0
Gr Liv Area               0
Kitchen Qual              0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
dtype: int64
There are some entries with null va

,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,6,1,3,3,1,0,4,1,0,0,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,6,1,3,4,0,1,4,1,0,0,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,6,1,3,3,1,0,3,1,0,1,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,1,1,3,4,0,1,3,1,0,0,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,1,1,4,2,0,1,3,1,0,0,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:
For BsmtFin SF 1, there is 1 null entry. Confirmed on website that this null entry is actually a house with no Basement, hence no BsmtFin SF 1. It should be 0.
Visual EDA indicates that BsmtFin SF 1 can be considered as predictor with correlation of 0.42
## Use BsmtFin SF 1 as Predictor, all Null values to be converted to 0.

# calling cleaning functions

df_under_cleaning=clean_condis_func (df_under_cleaning,'BsmtFin SF 1','Yes')

print(df_under_cleaning.shape)
df_under_cleaning.head()

In [28]:
# calling cleaning functions

df_under_cleaning=clean_condis_func (df_under_cleaning,'BsmtFin SF 1','No')

print(df_under_cleaning.shape)
df_under_cleaning.head()

BsmtFin Type 1            0
Bsmt Exposure             0
Bsmt Cond                 0
Bsmt Qual                 0
Foundation_CBlock         0
Foundation_PConc          0
Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
1st Flr SF                0
2nd Flr SF                0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
BsmtFin SF 1              1
Total Bsmt SF             1
Heating QC                0
Gr Liv Area               0
Kitchen Qual              0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
dtype: int64
There are some entries with null va

,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,6,1,3,3,1,0,4,1,0,0,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,6,1,3,4,0,1,4,1,0,0,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,6,1,3,3,1,0,3,1,0,1,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,1,1,3,4,0,1,3,1,0,0,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,1,1,4,2,0,1,3,1,0,0,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


In [29]:
# further cleaning BsmtFin SF 1

df_under_cleaning['BsmtFin SF 1'].replace('Unknown', 0,inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()

(2051, 34)


,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,6,1,3,3,1,0,4,1,0,0,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,6,1,3,4,0,1,4,1,0,0,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,6,1,3,3,1,0,3,1,0,1,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,1,1,3,4,0,1,3,1,0,0,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,1,1,4,2,0,1,3,1,0,0,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

For Total Bsmt SF, there is 1 null entry. Confirmed on website that this null entry is actually a house with no Basement, hence no Total Bsmt SF. It should be 0.

Correlation is 0.63 and visually strong.

## Use Total Bsmt SF  as predictor.

In [30]:
# calling cleaning functions

df_under_cleaning=clean_condis_func (df_under_cleaning,'Total Bsmt SF','No')

print(df_under_cleaning.shape)
df_under_cleaning.head()

BsmtFin Type 1            0
Bsmt Exposure             0
Bsmt Cond                 0
Bsmt Qual                 0
Foundation_CBlock         0
Foundation_PConc          0
Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
1st Flr SF                0
2nd Flr SF                0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
BsmtFin SF 1              0
Total Bsmt SF             1
Heating QC                0
Gr Liv Area               0
Kitchen Qual              0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
dtype: int64
There are some entries with null va

,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,6,1,3,3,1,0,4,1,0,0,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,6,1,3,4,0,1,4,1,0,0,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,6,1,3,3,1,0,3,1,0,1,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,1,1,3,4,0,1,3,1,0,0,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,1,1,4,2,0,1,3,1,0,0,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


In [31]:
# further cleaning Total Bsmt SF

df_under_cleaning['Total Bsmt SF'].replace('Unknown', 0,inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()

(2051, 34)


,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,Roof Style_Hip,Neighborhood_NAmes,...,Heating QC,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,6,1,3,3,1,0,4,1,0,0,...,Ex,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,6,1,3,4,0,1,4,1,0,0,...,Ex,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,6,1,3,3,1,0,3,1,0,1,...,TA,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,1,1,3,4,0,1,3,1,0,0,...,Gd,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,1,1,4,2,0,1,3,1,0,0,...,TA,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

Heating QC has no null values.

Seemingly good correlation of 0.46.

## Use Heating QC as Predictor.

In [32]:
# calling cleaning functions

df_under_cleaning=clean_ordinal_func (df_under_cleaning,'Heating QC','Yes','Unknown,Po,Fa,TA,Gd,Ex')

print(df_under_cleaning.shape)
df_under_cleaning.head()

BsmtFin Type 1            0
Bsmt Exposure             0
Bsmt Cond                 0
Bsmt Qual                 0
Foundation_CBlock         0
Foundation_PConc          0
Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
1st Flr SF                0
2nd Flr SF                0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
BsmtFin SF 1              0
Total Bsmt SF             0
Heating QC                0
Gr Liv Area               0
Kitchen Qual              0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
dtype: int64
There are no entries with null valu

,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,Roof Style_Hip,...,Total Bsmt SF,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice
0,5,6,1,3,3,1,0,4,1,0,...,725.0,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500
1,5,6,1,3,4,0,1,4,1,0,...,913.0,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000
2,3,6,1,3,3,1,0,3,1,0,...,1057.0,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000
3,4,1,1,3,4,0,1,3,1,0,...,384.0,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000
4,3,1,1,4,2,0,1,3,1,0,...,676.0,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500


## ANS:

Total Flr SF has no null values.

Correlation is very good at 0.71, even better than 1st Flr SF.

Visually fairly normally distributed, strong predictor.

apply this feature engineering to holdout dataset as well.

## Use Total Flr SF as predictor.

In [33]:
# Code to feature engineer 1st Flr SF and 2nd Flr SF into Total Flr SF

df_under_cleaning['Total Flr SF']=df_under_cleaning['1st Flr SF']+df_under_cleaning['2nd Flr SF']

df_under_cleaning.drop(columns=['1st Flr SF','2nd Flr SF'],inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()


(2051, 33)


,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,Roof Style_Hip,...,Gr Liv Area,Kitchen Qual,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,5,6,1,3,3,1,0,4,1,0,...,1479,Gd,6,Attchd,1976.0,RFn,2.0,475.0,130500,1479
1,5,6,1,3,4,0,1,4,1,0,...,2122,Gd,8,Attchd,1997.0,RFn,2.0,559.0,220000,2122
2,3,6,1,3,3,1,0,3,1,0,...,1057,Gd,5,Detchd,1953.0,Unf,1.0,246.0,109000,1057
3,4,1,1,3,4,0,1,3,1,0,...,1444,TA,7,BuiltIn,2007.0,Fin,2.0,400.0,174000,1444
4,3,1,1,4,2,0,1,3,1,0,...,1445,TA,6,Detchd,1957.0,Unf,2.0,484.0,138500,1445


## ANS:

Gr Liv Area has no null values.

Correlation is very good at 0.7.

Visually fairly normally distributed, strong predictor.

apply this feature engineering to holdout dataset as well.

## Use Gr Liv Area as predictor.

## ANS:

Kitchen Qual has no null values.
Good distribution, good correlation of 0.69.

## Use Kitchen Qual as Predictor.

In [34]:
# calling cleaning functions

df_under_cleaning=clean_ordinal_func (df_under_cleaning,'Kitchen Qual','Yes','Unknown,Po,Fa,TA,Gd,Ex')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Heating QC                0
BsmtFin Type 1            0
Bsmt Exposure             0
Bsmt Cond                 0
Bsmt Qual                 0
Foundation_CBlock         0
Foundation_PConc          0
Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
BsmtFin SF 1              0
Total Bsmt SF             0
Gr Liv Area               0
Kitchen Qual              0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
Total Flr SF              0
dtype: int64
There are no entries with null values in 'Kitchen Qual'.

These

,Kitchen Qual,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,Exter Qual,Roof Style_Gable,...,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,4,5,6,1,3,3,1,0,4,1,...,725.0,1479,6,Attchd,1976.0,RFn,2.0,475.0,130500,1479
1,4,5,6,1,3,4,0,1,4,1,...,913.0,2122,8,Attchd,1997.0,RFn,2.0,559.0,220000,2122
2,4,3,6,1,3,3,1,0,3,1,...,1057.0,1057,5,Detchd,1953.0,Unf,1.0,246.0,109000,1057
3,3,4,1,1,3,4,0,1,3,1,...,384.0,1444,7,BuiltIn,2007.0,Fin,2.0,400.0,174000,1444
4,3,3,1,1,4,2,0,1,3,1,...,676.0,1445,6,Detchd,1957.0,Unf,2.0,484.0,138500,1445


## ANS:

TotRms AbvGrd has no null values.
Good distribution, good correlation of 0.5.

## Use TotRms AbvGrd as Predictor.

## ANS:

## Due to high numbers and correlation of Detchd and Attchd, take these 2 Garage Types as predictors, the correlation seems logical too.


Garage Type has 113 null values indicating no garage. Replacing null with Unknown in dummy_variable column, then removing it. Will do the same for holdout data.


In [35]:
# calling cleaning functions

df_under_cleaning=clean_nominal_func (df_under_cleaning,'Garage Type','No')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Kitchen Qual              0
Heating QC                0
BsmtFin Type 1            0
Bsmt Exposure             0
Bsmt Cond                 0
Bsmt Qual                 0
Foundation_CBlock         0
Foundation_PConc          0
Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
BsmtFin SF 1              0
Total Bsmt SF             0
Gr Liv Area               0
TotRms AbvGrd             0
Garage Type             113
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
Total Flr SF              0
dtype: int64
There are some entries with null values in 'Garage Type'.


Aft

,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_Unknown,Kitchen Qual,Heating QC,BsmtFin Type 1,...,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,0,1,0,0,0,0,0,4,5,6,...,533.0,725.0,1479,6,1976.0,RFn,2.0,475.0,130500,1479
1,0,1,0,0,0,0,0,4,5,6,...,637.0,913.0,2122,8,1997.0,RFn,2.0,559.0,220000,2122
2,0,0,0,0,0,1,0,4,3,6,...,731.0,1057.0,1057,5,1953.0,Unf,1.0,246.0,109000,1057
3,0,0,0,1,0,0,0,3,4,1,...,0.0,384.0,1444,7,2007.0,Fin,2.0,400.0,174000,1444
4,0,0,0,0,0,1,0,3,3,1,...,0.0,676.0,1445,6,1957.0,Unf,2.0,484.0,138500,1445


In [36]:
# further cleaning Garage Type

df_under_cleaning.drop(columns=['Garage Type_2Types', 'Garage Type_Basment',
                                'Garage Type_BuiltIn', 'Garage Type_CarPort',
                                'Garage Type_Unknown'],inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()

(2051, 34)


,Garage Type_Attchd,Garage Type_Detchd,Kitchen Qual,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,...,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,1,0,4,5,6,1,3,3,1,0,...,533.0,725.0,1479,6,1976.0,RFn,2.0,475.0,130500,1479
1,1,0,4,5,6,1,3,4,0,1,...,637.0,913.0,2122,8,1997.0,RFn,2.0,559.0,220000,2122
2,0,1,4,3,6,1,3,3,1,0,...,731.0,1057.0,1057,5,1953.0,Unf,1.0,246.0,109000,1057
3,0,0,3,4,1,1,3,4,0,1,...,0.0,384.0,1444,7,2007.0,Fin,2.0,400.0,174000,1444
4,0,1,3,3,1,1,4,2,0,1,...,0.0,676.0,1445,6,1957.0,Unf,2.0,484.0,138500,1445


## ANS:

From web sourced data, garage yr blt is actually 2007 instead of 2207, for house Id 2261. Data was amended to change this typo.

function then called again.

Correlation is decent at 0.54. However, I do not consider garage yr blt as important as year built, when garage blt's correlation with SalePrice is as a tag along to year built.

114 null values.

##  Use Garage Yr Blt as predictor. impute all NaN values with the Year Blt.

In [37]:
# calling cleaning functions

df_under_cleaning=clean_condis_func (df_under_cleaning,'Garage Yr Blt','No')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Garage Type_Attchd        0
Garage Type_Detchd        0
Kitchen Qual              0
Heating QC                0
BsmtFin Type 1            0
Bsmt Exposure             0
Bsmt Cond                 0
Bsmt Qual                 0
Foundation_CBlock         0
Foundation_PConc          0
Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
BsmtFin SF 1              0
Total Bsmt SF             0
Gr Liv Area               0
TotRms AbvGrd             0
Garage Yr Blt           114
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
Total Flr SF              0
dtype: int64
There are some entries with null va

,Garage Type_Attchd,Garage Type_Detchd,Kitchen Qual,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,...,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,1,0,4,5,6,1,3,3,1,0,...,533.0,725.0,1479,6,1976,RFn,2.0,475.0,130500,1479
1,1,0,4,5,6,1,3,4,0,1,...,637.0,913.0,2122,8,1997,RFn,2.0,559.0,220000,2122
2,0,1,4,3,6,1,3,3,1,0,...,731.0,1057.0,1057,5,1953,Unf,1.0,246.0,109000,1057
3,0,0,3,4,1,1,3,4,0,1,...,0.0,384.0,1444,7,2007,Fin,2.0,400.0,174000,1444
4,0,1,3,3,1,1,4,2,0,1,...,0.0,676.0,1445,6,1957,Unf,2.0,484.0,138500,1445


In [38]:
# further cleaning Garage Yr Blt

df_under_cleaning['Garage Yr Blt'].replace('Unknown', df_under_cleaning['Year Built'],inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()

(2051, 34)


,Garage Type_Attchd,Garage Type_Detchd,Kitchen Qual,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,Foundation_PConc,...,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,1,0,4,5,6,1,3,3,1,0,...,533.0,725.0,1479,6,1976.0,RFn,2.0,475.0,130500,1479
1,1,0,4,5,6,1,3,4,0,1,...,637.0,913.0,2122,8,1997.0,RFn,2.0,559.0,220000,2122
2,0,1,4,3,6,1,3,3,1,0,...,731.0,1057.0,1057,5,1953.0,Unf,1.0,246.0,109000,1057
3,0,0,3,4,1,1,3,4,0,1,...,0.0,384.0,1444,7,2007.0,Fin,2.0,400.0,174000,1444
4,0,1,3,3,1,1,4,2,0,1,...,0.0,676.0,1445,6,1957.0,Unf,2.0,484.0,138500,1445


## ANS:

Correlation is decent at 0.53. 

114 null values.

##  Use Garage Finish as predictor. impute all NaN values with the 2.0 as median.

In [39]:
# calling cleaning functions

df_under_cleaning=clean_ordinal_func (df_under_cleaning,'Garage Finish','No','Unknown,Unf,RFn,Fin')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Garage Type_Attchd        0
Garage Type_Detchd        0
Kitchen Qual              0
Heating QC                0
BsmtFin Type 1            0
Bsmt Exposure             0
Bsmt Cond                 0
Bsmt Qual                 0
Foundation_CBlock         0
Foundation_PConc          0
Exter Qual                0
Roof Style_Gable          0
Roof Style_Hip            0
Neighborhood_NAmes        0
Neighborhood_NoRidge      0
Neighborhood_Somerst      0
MS Zoning                 0
MS SubClass_50            0
MS SubClass_60            0
Lot Frontage              0
Lot Area                  0
Overall Qual              0
Year Built                0
Mas Vnr Type             22
BsmtFin SF 1              0
Total Bsmt SF             0
Gr Liv Area               0
TotRms AbvGrd             0
Garage Yr Blt             0
Garage Finish           114
Garage Cars               1
Garage Area               1
SalePrice                 0
Total Flr SF              0
dtype: int64
There are some entries with null va

,Garage Finish,Garage Type_Attchd,Garage Type_Detchd,Kitchen Qual,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,...,Mas Vnr Type,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,2,1,0,4,5,6,1,3,3,1,...,BrkFace,533.0,725.0,1479,6,1976.0,2.0,475.0,130500,1479
1,2,1,0,4,5,6,1,3,4,0,...,BrkFace,637.0,913.0,2122,8,1997.0,2.0,559.0,220000,2122
2,1,0,1,4,3,6,1,3,3,1,...,None,731.0,1057.0,1057,5,1953.0,1.0,246.0,109000,1057
3,3,0,0,3,4,1,1,3,4,0,...,None,0.0,384.0,1444,7,2007.0,2.0,400.0,174000,1444
4,1,0,1,3,3,1,1,4,2,0,...,None,0.0,676.0,1445,6,1957.0,2.0,484.0,138500,1445


In [40]:
# further cleaning Garage Finish

df_under_cleaning['Garage Finish'].replace(0, 2,inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()

(2051, 34)


,Garage Finish,Garage Type_Attchd,Garage Type_Detchd,Kitchen Qual,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,...,Mas Vnr Type,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,2,1,0,4,5,6,1,3,3,1,...,BrkFace,533.0,725.0,1479,6,1976.0,2.0,475.0,130500,1479
1,2,1,0,4,5,6,1,3,4,0,...,BrkFace,637.0,913.0,2122,8,1997.0,2.0,559.0,220000,2122
2,1,0,1,4,3,6,1,3,3,1,...,None,731.0,1057.0,1057,5,1953.0,1.0,246.0,109000,1057
3,3,0,0,3,4,1,1,3,4,0,...,None,0.0,384.0,1444,7,2007.0,2.0,400.0,174000,1444
4,1,0,1,3,3,1,1,4,2,0,...,None,0.0,676.0,1445,6,1957.0,2.0,484.0,138500,1445


## ANS:

Correlation is good at 0.65. 

1 null values.

##  Use Garage Cars as predictor. impute all NaN values with 0, as no garage.

In [41]:
# calling cleaning functions

df_under_cleaning=clean_condis_func (df_under_cleaning,'Garage Cars','No')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Garage Finish            0
Garage Type_Attchd       0
Garage Type_Detchd       0
Kitchen Qual             0
Heating QC               0
BsmtFin Type 1           0
Bsmt Exposure            0
Bsmt Cond                0
Bsmt Qual                0
Foundation_CBlock        0
Foundation_PConc         0
Exter Qual               0
Roof Style_Gable         0
Roof Style_Hip           0
Neighborhood_NAmes       0
Neighborhood_NoRidge     0
Neighborhood_Somerst     0
MS Zoning                0
MS SubClass_50           0
MS SubClass_60           0
Lot Frontage             0
Lot Area                 0
Overall Qual             0
Year Built               0
Mas Vnr Type            22
BsmtFin SF 1             0
Total Bsmt SF            0
Gr Liv Area              0
TotRms AbvGrd            0
Garage Yr Blt            0
Garage Cars              1
Garage Area              1
SalePrice                0
Total Flr SF             0
dtype: int64
There are some entries with null values in 'Garage Cars'.


After nul

,Garage Finish,Garage Type_Attchd,Garage Type_Detchd,Kitchen Qual,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,...,Mas Vnr Type,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,2,1,0,4,5,6,1,3,3,1,...,BrkFace,533.0,725.0,1479,6,1976.0,2,475.0,130500,1479
1,2,1,0,4,5,6,1,3,4,0,...,BrkFace,637.0,913.0,2122,8,1997.0,2,559.0,220000,2122
2,1,0,1,4,3,6,1,3,3,1,...,None,731.0,1057.0,1057,5,1953.0,1,246.0,109000,1057
3,3,0,0,3,4,1,1,3,4,0,...,None,0.0,384.0,1444,7,2007.0,2,400.0,174000,1444
4,1,0,1,3,3,1,1,4,2,0,...,None,0.0,676.0,1445,6,1957.0,2,484.0,138500,1445


In [42]:
# further cleaning Garage Cars

df_under_cleaning['Garage Cars'].replace('Unknown', 0,inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()

(2051, 34)


,Garage Finish,Garage Type_Attchd,Garage Type_Detchd,Kitchen Qual,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,...,Mas Vnr Type,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,2,1,0,4,5,6,1,3,3,1,...,BrkFace,533.0,725.0,1479,6,1976.0,2.0,475.0,130500,1479
1,2,1,0,4,5,6,1,3,4,0,...,BrkFace,637.0,913.0,2122,8,1997.0,2.0,559.0,220000,2122
2,1,0,1,4,3,6,1,3,3,1,...,None,731.0,1057.0,1057,5,1953.0,1.0,246.0,109000,1057
3,3,0,0,3,4,1,1,3,4,0,...,None,0.0,384.0,1444,7,2007.0,2.0,400.0,174000,1444
4,1,0,1,3,3,1,1,4,2,0,...,None,0.0,676.0,1445,6,1957.0,2.0,484.0,138500,1445


## ANS:

Correlation is good at 0.65. 

114 null values.

##  Use Garage Area as predictor. impute all NaN values with 0, as no garage.

In [43]:
# calling cleaning functions

df_under_cleaning=clean_condis_func (df_under_cleaning,'Garage Area','No')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Garage Finish            0
Garage Type_Attchd       0
Garage Type_Detchd       0
Kitchen Qual             0
Heating QC               0
BsmtFin Type 1           0
Bsmt Exposure            0
Bsmt Cond                0
Bsmt Qual                0
Foundation_CBlock        0
Foundation_PConc         0
Exter Qual               0
Roof Style_Gable         0
Roof Style_Hip           0
Neighborhood_NAmes       0
Neighborhood_NoRidge     0
Neighborhood_Somerst     0
MS Zoning                0
MS SubClass_50           0
MS SubClass_60           0
Lot Frontage             0
Lot Area                 0
Overall Qual             0
Year Built               0
Mas Vnr Type            22
BsmtFin SF 1             0
Total Bsmt SF            0
Gr Liv Area              0
TotRms AbvGrd            0
Garage Yr Blt            0
Garage Cars              0
Garage Area              1
SalePrice                0
Total Flr SF             0
dtype: int64
There are some entries with null values in 'Garage Area'.


After nul

,Garage Finish,Garage Type_Attchd,Garage Type_Detchd,Kitchen Qual,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,...,Mas Vnr Type,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,2,1,0,4,5,6,1,3,3,1,...,BrkFace,533.0,725.0,1479,6,1976.0,2.0,475,130500,1479
1,2,1,0,4,5,6,1,3,4,0,...,BrkFace,637.0,913.0,2122,8,1997.0,2.0,559,220000,2122
2,1,0,1,4,3,6,1,3,3,1,...,None,731.0,1057.0,1057,5,1953.0,1.0,246,109000,1057
3,3,0,0,3,4,1,1,3,4,0,...,None,0.0,384.0,1444,7,2007.0,2.0,400,174000,1444
4,1,0,1,3,3,1,1,4,2,0,...,None,0.0,676.0,1445,6,1957.0,2.0,484,138500,1445


In [44]:
# further cleaning Garage Area

df_under_cleaning['Garage Area'].replace('Unknown', 0,inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()

(2051, 34)


,Garage Finish,Garage Type_Attchd,Garage Type_Detchd,Kitchen Qual,Heating QC,BsmtFin Type 1,Bsmt Exposure,Bsmt Cond,Bsmt Qual,Foundation_CBlock,...,Mas Vnr Type,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,2,1,0,4,5,6,1,3,3,1,...,BrkFace,533.0,725.0,1479,6,1976.0,2.0,475.0,130500,1479
1,2,1,0,4,5,6,1,3,4,0,...,BrkFace,637.0,913.0,2122,8,1997.0,2.0,559.0,220000,2122
2,1,0,1,4,3,6,1,3,3,1,...,None,731.0,1057.0,1057,5,1953.0,1.0,246.0,109000,1057
3,3,0,0,3,4,1,1,3,4,0,...,None,0.0,384.0,1444,7,2007.0,2.0,400.0,174000,1444
4,1,0,1,3,3,1,1,4,2,0,...,None,0.0,676.0,1445,6,1957.0,2.0,484.0,138500,1445


## ANS:

For Mas Vnr Type, there are 22 null entries.

1218/2051 have None for Mas Vnr Type, with a correlation of -0.42.

Note that holdout data has 1 null entry for Mas Vnr Type. if not dropping this entire column, need to treat same for both training and holdout.

BrkFace is 630/2051, correlation of 0.27.
Stone is 168/2051, correlation of 0.31.

checked the lone null entry on holdout data, and the PID of the house when referenced against the source data provided via http://www.cityofames.org/assessor/ website, it should be Mas Vnr Type = None.

## Consider using Mas Vnr Type None, BrkFace and Stone as predictors. The 22 Null entries are likely to be dropped as row entries, as we have no efficient and accurate method of determining the 22 Null entries for training data Mas Vnr Type.

Will split into dummy_variable columns and drop all Max Vnr Type Unknown columns for both training and holdout data.


In [45]:
# calling cleaning functions

df_under_cleaning=clean_nominal_func (df_under_cleaning,'Mas Vnr Type','No')

print(df_under_cleaning.shape)
df_under_cleaning.head()

Garage Finish            0
Garage Type_Attchd       0
Garage Type_Detchd       0
Kitchen Qual             0
Heating QC               0
BsmtFin Type 1           0
Bsmt Exposure            0
Bsmt Cond                0
Bsmt Qual                0
Foundation_CBlock        0
Foundation_PConc         0
Exter Qual               0
Roof Style_Gable         0
Roof Style_Hip           0
Neighborhood_NAmes       0
Neighborhood_NoRidge     0
Neighborhood_Somerst     0
MS Zoning                0
MS SubClass_50           0
MS SubClass_60           0
Lot Frontage             0
Lot Area                 0
Overall Qual             0
Year Built               0
Mas Vnr Type            22
BsmtFin SF 1             0
Total Bsmt SF            0
Gr Liv Area              0
TotRms AbvGrd            0
Garage Yr Blt            0
Garage Cars              0
Garage Area              0
SalePrice                0
Total Flr SF             0
dtype: int64
There are some entries with null values in 'Mas Vnr Type'.


After nu

,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_None,Mas Vnr Type_Stone,Mas Vnr Type_Unknown,Garage Finish,Garage Type_Attchd,Garage Type_Detchd,Kitchen Qual,Heating QC,...,Year Built,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,0,1,0,0,0,2,1,0,4,5,...,1976,533.0,725.0,1479,6,1976.0,2.0,475.0,130500,1479
1,0,1,0,0,0,2,1,0,4,5,...,1996,637.0,913.0,2122,8,1997.0,2.0,559.0,220000,2122
2,0,0,1,0,0,1,0,1,4,3,...,1953,731.0,1057.0,1057,5,1953.0,1.0,246.0,109000,1057
3,0,0,1,0,0,3,0,0,3,4,...,2006,0.0,384.0,1444,7,2007.0,2.0,400.0,174000,1444
4,0,0,1,0,0,1,0,1,3,3,...,1900,0.0,676.0,1445,6,1957.0,2.0,484.0,138500,1445


In [46]:
# further cleaning Mas Vnr Type

df_under_cleaning.drop(columns=['Mas Vnr Type_BrkCmn', 'Mas Vnr Type_Unknown'],inplace=True)

print(df_under_cleaning.shape)
df_under_cleaning.head()



(2051, 36)


,Mas Vnr Type_BrkFace,Mas Vnr Type_None,Mas Vnr Type_Stone,Garage Finish,Garage Type_Attchd,Garage Type_Detchd,Kitchen Qual,Heating QC,BsmtFin Type 1,Bsmt Exposure,...,Year Built,BsmtFin SF 1,Total Bsmt SF,Gr Liv Area,TotRms AbvGrd,Garage Yr Blt,Garage Cars,Garage Area,SalePrice,Total Flr SF
0,1,0,0,2,1,0,4,5,6,1,...,1976,533.0,725.0,1479,6,1976.0,2.0,475.0,130500,1479
1,1,0,0,2,1,0,4,5,6,1,...,1996,637.0,913.0,2122,8,1997.0,2.0,559.0,220000,2122
2,0,1,0,1,0,1,4,3,6,1,...,1953,731.0,1057.0,1057,5,1953.0,1.0,246.0,109000,1057
3,0,1,0,3,0,0,3,4,1,1,...,2006,0.0,384.0,1444,7,2007.0,2.0,400.0,174000,1444
4,0,1,0,1,0,1,3,3,1,1,...,1900,0.0,676.0,1445,6,1957.0,2.0,484.0,138500,1445


In [47]:
df_under_cleaning.isnull().sum()

Mas Vnr Type_BrkFace    0
Mas Vnr Type_None       0
Mas Vnr Type_Stone      0
Garage Finish           0
Garage Type_Attchd      0
Garage Type_Detchd      0
Kitchen Qual            0
Heating QC              0
BsmtFin Type 1          0
Bsmt Exposure           0
Bsmt Cond               0
Bsmt Qual               0
Foundation_CBlock       0
Foundation_PConc        0
Exter Qual              0
Roof Style_Gable        0
Roof Style_Hip          0
Neighborhood_NAmes      0
Neighborhood_NoRidge    0
Neighborhood_Somerst    0
MS Zoning               0
MS SubClass_50          0
MS SubClass_60          0
Lot Frontage            0
Lot Area                0
Overall Qual            0
Year Built              0
BsmtFin SF 1            0
Total Bsmt SF           0
Gr Liv Area             0
TotRms AbvGrd           0
Garage Yr Blt           0
Garage Cars             0
Garage Area             0
SalePrice               0
Total Flr SF            0
dtype: int64

In [48]:
# forcing NaN values for any errors as last check
df_under_cleaning = df_under_cleaning.apply(pd.to_numeric, errors='coerce')

In [49]:
df_under_cleaning.isnull().sum()

Mas Vnr Type_BrkFace    0
Mas Vnr Type_None       0
Mas Vnr Type_Stone      0
Garage Finish           0
Garage Type_Attchd      0
Garage Type_Detchd      0
Kitchen Qual            0
Heating QC              0
BsmtFin Type 1          0
Bsmt Exposure           0
Bsmt Cond               0
Bsmt Qual               0
Foundation_CBlock       0
Foundation_PConc        0
Exter Qual              0
Roof Style_Gable        0
Roof Style_Hip          0
Neighborhood_NAmes      0
Neighborhood_NoRidge    0
Neighborhood_Somerst    0
MS Zoning               0
MS SubClass_50          0
MS SubClass_60          0
Lot Frontage            0
Lot Area                0
Overall Qual            0
Year Built              0
BsmtFin SF 1            0
Total Bsmt SF           0
Gr Liv Area             0
TotRms AbvGrd           0
Garage Yr Blt           0
Garage Cars             0
Garage Area             0
SalePrice               0
Total Flr SF            0
dtype: int64

In [50]:
df_traindata_cleaned= pd.DataFrame.copy(df_under_cleaning)

In [51]:
df_traindata_cleaned.to_csv(r'../datasets/df_traindata_cleaned.csv', index = False)

In [52]:
## End of File